In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
import analyze_covid
import airport_data_explore

# Unload stored dataframes from other notebooks within directory
%store -r

/home/aytang/sfuhome/353project/airport_data_explore.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  canada['Date'] = pd.to_datetime(canada['Date'])


In [2]:
# Haversine Caluclation (Distance between two lat and lon coordinates) retrieved from:
# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula/21623206

def haversine(lat1, lon1, lat2, lon2):
    p = np.pi / 180
    a = 0.5 - np.cos((lat2 - lat1) * p) / 2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p )) / 2
    n = 12742 * np.arcsin(np.sqrt(a))
    return n  

In [41]:
def getStates():
    return usLocations['Province/State'].unique()

In [4]:
def closest(state):
    d = []
    
    t = usLocations[usLocations["Province/State"] == state]
    i = airportLocationsUS[airportLocationsUS["State"] == state]
    size = len(t)
    size2 = len(i)

    for a in range(size2):
        for s in range(size):
            airport = {
                    'country': t.iloc[s]["Country/Region"],
                    'state/province': t.iloc[s]["Province/State"],
                    'county': t.iloc[s]["County"],
                    'lat': t.iloc[s]["Latitude"],
                    'long': t.iloc[s]["Longitude"],
                    'airport': i.iloc[a]["AirportName"],
                    'distance(km)': haversine(t.iloc[s]["Latitude"], t.iloc[s]["Longitude"], i.iloc[a]["lat"], i.iloc[a]["long"])
                }
            d.append(airport)
    d = sorted(d, key = lambda x: x['distance(km)'])
    return d

In [5]:
def calcNearestAirport(state):
    d = []
    for s in state:
        d.append(closest(state))
    return d[0]

#calcNearestAirport('California')        

In [26]:
statesList = getStates()
listDf = []

for s in statesList:
    i = calcNearestAirport(s)
    listDf.append(pd.DataFrame.from_dict(i))
    
main = pd.concat(listDf)

In [43]:
listDf2 = []

for s in statesList:
    df = main[main["state/province"] == s].sort_values(by = ['county', 'distance(km)']).reset_index(drop = True)
    df = df.iloc[::2]
    listDf2.append(df)

usa = pd.concat(listDf2).reset_index(drop = True)
usa
# Done. Figure out what to do with this next. 
# Group by airport? i.e. LaGuardia: Every county in NY except Kings and Nassau

,airport,country,county,distance(km),lat,long,state/province
0,LaGuardia,US,Albany,203.305377,42.600603,-73.977239,New York
1,LaGuardia,US,Allegany,383.150746,42.257484,-78.027505,New York
2,LaGuardia,US,Bronx,8.740585,40.852093,-73.862828,New York
3,LaGuardia,US,Broome,223.265360,42.159032,-75.813261,New York
4,LaGuardia,US,Cattaraugus,432.353848,42.247782,-78.679231,New York
5,LaGuardia,US,Cayuga,325.527992,42.912617,-76.557316,New York
6,LaGuardia,US,Chautauqua,485.116600,42.227692,-79.366918,New York
7,LaGuardia,US,Chemung,284.699688,42.138911,-76.763880,New York
8,LaGuardia,US,Chenango,239.575861,42.494300,-75.608876,New York
9,LaGuardia,US,Clinton,441.887337,44.745309,-73.678754,New York
